# NSLS-II: Image segmentation workflow example

This notebook serves as a general illustration of the steps and tools required in order to segment three-dimensional image data sets for quantitative analysis. 

The demonstration steps through a sample segmentation workflow, specific to x-ray tomography, and is broken down into each of the intermediate steps required: from "raw" collected image to quantified values and parameters.

## Sample Dataset

The sample data sets consist of micro-CT volumes collected using a single glass bead packed column.

The final quantified product in this demonstration includes:
- Solid volume (total volume of the beads)
- Measured porosity
- Surface area
- Curvature
- Solid grain size distribution (comparison of the volume and surface area of the individual beads in the column section)

### Global Dependencies:

The major python packages that are utilized in order to provide image processing functionality, quantification, and visualization include:
+ Reconstruction
  - tomopy
+ Registration
  - SimpleITK
+ Analysis:
  - Numpy
  - Scikit-Image
  - Scikit-Xray
  
+ Visualization:
  - Summary Tables
    - Pandas
  - Two-dimensional Visualization
    - Xray-vision
    - Matplotlib
    - Pyqtgraph
  - Three-dimensional Visualization
    - VTK
    - MayaVI
    - Pyqtgraph

These packages are imported here for common use throughout the demonstration.

An equivalent and parallel demonstration, using VisTrails, is provided to illustrate how to conduct the same operations using the VisTrails workflow and provanance manager.

**In the current implementation both mayavi and xray-vision must be imported first, in order to prevent conflicts with default settings associated with direct import of matplotlib.**

In [1]:
import sip
sip.setapi('QString', 2)
import mayavi.mlab as mlab
from xray_vision.mpl_plotting.img_proc_tools import plot_multiple_dsets


********************************************************************************
         to build the TVTK classes (5.1). This may cause problems.
         Please rebuild TVTK.
********************************************************************************



In [2]:
# File I/O
#from skxray import io
from vttools.to_wrap.image.io import read_tiff
#import pims
import tifffile

In [3]:
# Tomographic reconstruction
import tomopy

In [4]:
# Image segmentation, analysis and quantification
import numpy as np
#import scipy as sp
import skxray
import skimage

One particularly nice feature of iPython Notebook is the ability to have graphical results be plotted inline with the text and code that comprise this demonstration. In order to enable inline plotting using matplotlib

```python
%matplotlib inline
```

is included as a 'magic' function prior to the import string for matplotlib.

In [5]:
# Plotting and visualization tools
%matplotlib inline
import matplotlib
matplotlib.rcParams["backend"] = "Qt4Agg"
import matplotlib.pyplot as plt
import pyqtgraph as pqg
import vtk
import cv2

# Reconstruction

Add reconstruction information and example here

USING TOMOPY

### List of camera files associated with the data sets

Note there are three .SPE camera files associated with each data set. The data was collected as part of a biofilm growth experiment conducted in collaboration with Dorthe Wildenschild. Imaging was conducted at the Advanced Photon Source, Argonne National Laboratory, Sector 13, BM-D.

The data was collected in March of 2012.

In [6]:
path_cam_files = "../data/mCT/camera_files/"
cam_fname_abv = ['biofilm_abv_cam_1.SPE', 'biofilm_abv_cam_2.SPE', 'biofilm_abv_cam_3.SPE']
cam_fname_blw = ['biofilm_blw_cam_1.SPE', 'biofilm_blw_cam_2.SPE', 'biofilm_blw_cam_3.SPE']

## Load raw camera (SPE) data for reconstruction

In [7]:
camfile_abv = []
camfile_blw = []
cam_lst = [camfile_abv, camfile_blw]
for index, dset_list in enumerate([cam_fname_abv, cam_fname_blw]):
    for fname in dset_list:
        cam_lst[index].append(tomopy.read_aps_13bm(path_cam_files + fname, 'spe'))


ERROR:tomopy.io.reader:unknown file extension
ERROR:tomopy.io.reader:unknown file extension
ERROR:tomopy.io.reader:unknown file extension
ERROR:tomopy.io.reader:unknown file extension
ERROR:tomopy.io.reader:unknown file extension
ERROR:tomopy.io.reader:unknown file extension


In [12]:
len(cam_lst)

2

In [13]:
len(cam_lst[0])

3

In [16]:
len(cam_lst[0][2])

10

In [ ]:
tmp

In [11]:
help(tomopy.read_spe)

Help on function read_spe in module tomopy.io.reader:

read_spe(fname, slc=None)
    Read data from spe file.
    
    Parameters
    ----------
    fname : str
        String defining the path or file name.
    slc : {sequence, int}
        Range of values for slicing data.
        ((start_1, end_1, step_1), ... , (start_N, end_N, step_N))
        defines slicing parameters for each axis of the data matrix.
    
    Returns
    -------
    ndarray
        Data.



In [10]:
help(tomopy.read_aps_13bm)

Help on function read_aps_13bm in module tomopy.io.exchange:

read_aps_13bm(fname, format, proj=None, sino=None)
    Read APS 13-BM standard data format.
    
    Parameters
    ----------
    fname : str
        Path to hdf5 file.
    
    format : str
        Data format. 'spe' or 'netcdf4'
    
    proj : {sequence, int}, optional
        Specify projections to read. (start, end, step)
    
    sino : {sequence, int}, optional
        Specify sinograms to read. (start, end, step)
    
    Returns
    -------
    ndarray
        3D tomographic data.



In [9]:
help(tomopy.find_center)

Help on function find_center in module tomopy.recon.rotation:

find_center(tomo, theta, ind=None, emission=True, init=None, tol=0.5, mask=True, ratio=1.0)
    Find rotation axis location.
    
    The function exploits systematic artifacts in reconstructed images
    due to shifts in the rotation center. It uses image entropy
    as the error metric and ''Nelder-Mead'' routine (of the scipy
    optimization module) as the optimizer :cite:`Donath:06`.
    
    Parameters
    ----------
    tomo : ndarray
        3D tomographic data.
    theta : array
        Projection angles in radian.
    ind : int, optional
        Index of the slice to be used for reconstruction.
    emission : bool, optional
        Determines whether data is emission or transmission type.
    init : float
        Initial guess for the center.
    tol : scalar
        Desired sub-pixel accuracy.
    mask : bool, optional
        If ``True``, apply a circular mask to the reconstructed image to
        limit the anal

In [ ]:
recon_abv = tomopy.find_center

In [8]:
camfile_abv
np.amax(camfile_abv[0])

4095

## Step 1: Import Above and Below Edge volumes:

### Module: PIMS

In [ ]:
#vol_abv = pims.TiffStack(fname_abv)
#vol_blw = pims.TiffStack(fname_blw)

### This module contains functions and tools for reading, writing and converting common image processing files and file types.

In [ ]:
ls

In [ ]:
data_path = "../data/"

In [ ]:
#fname_abv_tiff = data_path + "uCT/NSLS_shale_smpl-2_AbvFe.tiff"
fname_blw_tiff = data_path + "uCT/NSLS_shale_smpl-2_BlwFe.tiff"
fname_abv = data_path + "mCT/biofilm_33pcntBa__A_recon.volume"
fname_blw = data_path + "mCT/biofilm_33pcntBa__B_recon.volume"
fname_bdpack = data_path + "mCT/dry_beadpack_A_recon.volume"


In [ ]:
ls ../data/uCT/

In [ ]:
pwd

In [ ]:
tst = read_tiff(fname_blw_tiff)

In [ ]:
fnames = [fname_abv, fname_blw, fname_bdpack]
#for index, var in enumerate(fnames):
#    print index
#    print var
header_abv, vol_abv = io.net_cdf_io.load_netCDF(fname_abv)
header_blw, vol_blw = io.net_cdf_io.load_netCDF(fname_blw)
header_dry, vol_dry = io.net_cdf_io.load_netCDF(fname_bdpack)

print "Array Max Value: " + str(np.max(vol_abv))
print "Array Min Value: " + str(np.min(vol_abv))
#print header_dict_abv


## Sample netCDF header
The header in these netCDF files retains important information pertinent to not only the data set, but also to how the data set was collected.

### Header Example:

In [ ]:
print header_blw

In [ ]:
#Select how many of the available volumes to load
num_vols = 3 # Length of the data list, dummy.
axis = "XY"
height = None
num_columns = None
#Input Lists
sample_data = [vol_abv, vol_blw, vol_dry]
plot_titles = ["Bead pack sample: above Ba edge", "Bead pack sample: below Ba edge", "Bead pack sample: Dry column"]
#Error: If other than XY selected and data is only 2D array, then raise error pointing this out.
plot_multiple_dsets (sample_data, 450, plot_titles, "YZ", num_columns=1)

    # if plot_titles=None:
        # for -- create for loop to generate list of str with (Dataset # 'index': 'dset variable name')
#for x in range(num_vols):
#    plt.figure(figsize = (5,(5.25*num_vols)))
#    num_rows = num_vols
#    num_columns = 1
#    plt.subplot(((num_rows * 10 + num_columns) * 10 + (x+1)))
#    plt.imshow(sample_data[x][:,450,:], cmap=plt.cm.gray)
#    plt.title(plot_titles[x])
#plt.show()

In [ ]:
import matplotlib.pyplot as plt
def auto_plot (num_figs, num_rows, num_cols, src_data_list, title_list=None, indi_figSize = None, display_slc=None):
    if indi_figSize == None:
        figsize = 5
    else:
        figsize = indi_figSize
    
    plt.figure(figsize = ((figsize*num_cols),(figsize*num_rows)))
    for x in range(len(src_data_list)):
        sample_data = src_data_list[x]
        if len(sample_data.shape) == 2:
            slc = sample_data[x][:,:]
        elif len(sample_data.shape) == 3 and display_slc == None:
            slc = sample_data[x][int(sample_data.shape[0]/2),:,:]
        elif len(sample_data.shape) == 3 and display_slc != None:
            slc = sample_data[x][int(display_slc),:,:]
        plt.subplot(((num_rows * 10 + num_columns) * 10 + (x+1)))
    plt.imshow(slc, cmap=plt.cm.gray)
    plt.title(title_list[x])
    plt.show()


## Step 2: Volume transformation to correct alignment

### Source Packages: vttools.to_wrap.image.transformation.py

### This module contains tools for basic spatial transformation of image volumes

In [ ]:
from vttools.to_wrap.image import transformation

The z-axis orientation for all three of the volumes loaded as part of this demonstration is flipped. A problem that isn't necessarily an issue unless spatial orientation is important in the data analysis (e.g. when looking at sequential tiles in a single experimental sample, or when looking at spatial changes in a material over the course of an experiment). As a result, the data sets need to be flipped along the z-axis.

In [ ]:
len(sample_data)

In [ ]:
corrected_src_data = []
for x in sample_data:
    xForm_vol = transformation.flip_axis(x, "Flip Z")
    corrected_src_data.append(xForm_vol)
num_rows=3
num_cols=2
plt.figure(figsize = ((5*num_cols),(5*num_rows)))
for x in range(len(corrected_src_data)):
    plt.subplot(((num_rows * 10 + (num_cols)) * 10 + 2*x+1))
    plt.subplot(320+(x*2+1))
    plt.imshow(sample_data[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Original grayscale data')
    plt.subplot(((num_rows * 10 + num_cols) * 10 + 2*x+2))
    plt.imshow(corrected_src_data[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Transformed grayscale data')

plt.show()

## Step 3: Histogram analysis of trial volume

This module consists of a collection of functions specific to histogram visualization and analysis. A variety of options are included for both displaying and saving histogram data.

### Module: histops.py

In [ ]:
from vttools.to_wrap.image import histogram
from xray_vision.mpl_plotting.img_proc_tools import plot_histogram

In [ ]:

num_cols = 2
num_rows = 3
#num_rows = num_vols

num_bins = 1500
pdf_hist = False
plt.figure(figsize = ((5.5*num_cols),(5*num_rows)))
plot_yscale = 'linear'
for x in range(len(corrected_src_data)):
    hist, bin_edges, bin_avg = histogram.hist_make(corrected_src_data[x], num_bins, pdf_hist)
    
    plot_min = np.amin(corrected_src_data[x])
    plot_max = np.amax(corrected_src_data[x])    
    plt.subplot(((num_rows * 10 + (num_cols)) * 10 + 2*x+1))
    plt.subplot(320+(x*2+1))
    plt.imshow(corrected_src_data[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Transformed grayscale data')
    plt.subplot(((num_rows * 10 + num_cols) * 10 + 2*x+2))
    #plot_histogram(hist, bin_edges, plot_min, plot_max, plot_yscale)
plt.show()
print np.amin(hist)
print np.amax(hist)

## Step 4: Thresholding for trial volume segmentation

### Module: threshops.py

### This module contains tools for thresholding image data.

In [ ]:
import threshops

In [ ]:
#Manual thresholding demonstration
thresh_value = 1000

solid_matrix = []

for x in range(len(corrected_src_data)):
    manual_result = threshops.thresh_globalGT(corrected_src_data[x], thresh_value)
    solid_matrix.append(manual_result)


In [ ]:
#Plotting manual thresholding results
num_cols = 3
num_rows = 3
#num_rows = num_vols

plt.figure(figsize = ((5.5*num_cols),(6*num_rows)))
for x in range(len(corrected_src_data)):
    hist, bin_edges, bin_avg = histops.hist_make(corrected_src_data[x], num_bins, pdf_hist)
    thresh_coords_x = [thresh_value, thresh_value]
    thresh_coords_y = [np.amin(hist), np.amax(hist)]
    plot_min = np.amin(corrected_src_data[x])
    plot_max = np.amax(corrected_src_data[x])    
    
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+1))
    plt.imshow(corrected_src_data[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Transformed grayscale data')
    
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+2))
    histops.hist_plot(hist, bin_edges, plot_min, plot_max, plot_yscale, 'No')
    plt.plot(thresh_coords_x, thresh_coords_y, color='r')
    plt.annotate('threshold marker', 
                 xy=(thresh_value, (np.amax(hist)*0.7)), 
                 xytext=((thresh_value+750), np.amax(hist)*0.95),
                 arrowprops=dict(facecolor='red', shrink=0.05)
                 )
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+3))
    plt.imshow(solid_matrix[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Binary result from thresholding')
    
plt.show()

In [ ]:
#Automatic thresholding demonstration
auto_adapt_solid = []
auto_otsu_solid = []
auto_yen_solid = []


for x in range(len(corrected_src_data)):
    
    adapt_block_size = 250
    adapt_thresh = threshops.thresh_adapt(corrected_src_data[x], adapt_block_size)
    auto_adapt_solid.append(adapt_thresh)
    
    otsu_thresh_trial, otsu_T_value = threshops.thresh_otsu(corrected_src_data[x])
    auto_otsu_solid.append(otsu_thresh_trial)
    
    yen_thresh_trial, yen_T_value = threshops.thresh_yen(corrected_src_data[x])
    auto_yen_solid.append(yen_thresh_trial)


In [ ]:
plt.imshow(solid_matrix[0][:,250,:], cmap=plt.cm.gray)
plt.show()
for x in range(num_vols):
    print x

In [ ]:
#Plotting automatic thresholding results
num_cols = 6
num_rows = 3
#num_rows = num_vols

plt.figure(figsize = ((5.5*num_cols),(6*num_rows)))
for x in range(num_vols):
    hist, bin_edges, bin_avg = histops.hist_make(corrected_src_data[x], num_bins, pdf_hist)
    
    thresh_type_labels = ['manual', 'Otsu', 'Yen']
    label_offsets = [0.95, 0.8, 0.6]
    thresh_coords_x = [[thresh_value, thresh_value], 
                       [otsu_T_value, otsu_T_value], 
                       [yen_T_value, yen_T_value]]
    thresh_coords_y = [np.amin(hist), np.amax(hist)]
    color_set_ = ['r', 'g', 'b']
    
    plot_min = np.amin(corrected_src_data[x])
    plot_max = np.amax(corrected_src_data[x])    
    
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+1))
    plt.imshow(corrected_src_data[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Transformed grayscale data')
    
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+2))
    histops.hist_plot(hist, bin_edges, plot_min, plot_max, plot_yscale, 'No')
    for y in range(len(thresh_type_labels)):
        plt.plot(thresh_coords_x[y], thresh_coords_y, color=color_set_[y])
        plt.annotate(thresh_type_labels[y], 
                     xy=(thresh_coords_x[y][0], (np.amax(hist)*0.7)),
                     xytext=((thresh_coords_x[y][0]+750), np.amax(hist)*label_offsets[y]), 
                     arrowprops=dict(facecolor=color_set_[y], shrink=0.05))
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+3))
    plt.imshow(solid_matrix[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Binary result from manual thresholding')
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+4))
    
    plt.imshow(auto_adapt_solid[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Binary result from Adaptive thresholding')
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+5))
    plt.imshow(auto_otsu_solid[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Binary result from Otsu thresholding')
    plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+6))
    plt.imshow(auto_yen_solid[x][:,250,:], cmap=plt.cm.gray)
    if x == 0:
        plt.title('Binary result from Yen thresholding')
plt.show()

### Adaptive thresholding: Block size evaluation

In [ ]:
ad_block_size = [5, 10, 20, 50, 100, 250]
num_cols = 6
num_rows = 3
#num_rows = num_vols

ad_test = []
plt.figure(figsize = (((5.5*num_cols),(6*num_rows))))
for x in range(len(corrected_src_data)):
    for y in range(len(ad_block_size)):
        tmp_binary = threshops.thresh_adapt(corrected_src_data[x], ad_block_size[y])
        ad_test.append(tmp_binary)
        if y == 0:
            plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+1))
            plt.imshow(corrected_src_data[x][:,250,:], cmap=plt.cm.gray)
            plt.title("Original Data")
        plt.subplot(((num_rows * 10 + num_cols) * 10 + num_cols*x+y+1))
        plt.imshow(ad_test[y][:,250,:])
        plt.title("Adaptive threshold: Block size = " + str(ad_block_size[y]))

plt.show()

## Step 5: Isolate pore space

### Module: mathops.py

In [ ]:
import mathops


In [ ]:
pore_space = []

for x in range(len(solid_matrix)):
    tmp_pore = np.logical_not(solid_matrix[x])
    pore_space.append(tmp_pore)


## Step 6: Generate exterior mask

### Module: synth_drawing

In [ ]:
import synth_drawing

In [ ]:

exterior_mask=synth_drawing.draw_cylinder(corrected_src_data[0], None, 325, None, None, 'YES')


#### Masked volume demonstration

In [ ]:
mask_demo=synth_drawing.draw_cylinder(corrected_src_data[0], 
                                      radius=325, 
                                      value = 2000, 
                                      draw_exterior='YES', 
                                      apply_mask='yes')

plt.imshow(mask_demo[50,:,:])
plt.show()

### Isolated phase presentation:

In [ ]:
#Glass beads: solid_matrix[0]
#Pore space: pore_space[0]
#exterior: exterior_mask



In [ ]:
plt.figure(figsize = (15,4.5))
plt.subplot(131)
plt.imshow(solid_matrix[0][50,:,:])
plt.subplot(132)
plt.imshow(pore_space[0][50,:,:])
plt.subplot(133)
plt.imshow(exterior_mask[50,:,:])
plt.show()

## Step 7: Merge materials and label

In [ ]:
import filterops as fltr
import morphology as morph
import scipy.ndimage.measurements as measure
import mathops

def logical_sub(src_data1, 
                src_data2):
    temp = np.logical_not(np.logical_and(src_data1, 
                                         src_data2))
    output = np.logical_and(src_data1, 
                            temp)
    return output



In [ ]:
#rough quantification
labels_rough = (exterior_mask) + (pore_space[0]) + (solid_matrix[0]*3)
glass_bead_corrected = 3*(logical_sub(solid_matrix[0], exterior_mask))
pore_space_corrected = 2*(logical_sub(pore_space[0], exterior_mask))

#Merge volume:
label_field = (exterior_mask + 2*pore_space_corrected + 3*glass_bead_corrected)


In [ ]:
label_field_int = label_field.astype(int)

In [ ]:
plt.imshow(label_field_int[50,:,:], cmap=plt.cm.gray)
plt.show()

In [ ]:
import imgQuant

measures, vol_rec = imgQuant.Q_VOL(label_field_int)

In [ ]:

bead_vol = measures['Material_4']['volume']['value']
pore_sp_vol = measures['Material_1']['volume']['value']

bead_vol_corrected = float(bead_vol) * 9.5**3
pore_sp_vol_corrected = float(pore_sp_vol) * 9.5**3
print "Measured bead volume (um^3): " + str(bead_vol_corrected)
print "Measured pore space (um^3): " + str(pore_sp_vol_corrected)
porosity = pore_sp_vol/(bead_vol+pore_sp_vol)
print "Measured column section porosity is: " + str(porosity)

In [ ]:
#Volume confirmation:
total_vol = bead_vol_corrected

In [ ]:
!gist "NSLS-II Segmentation workflow design.ipynb"